In [2]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd

PATH = os.getcwd()
PATH

'/home/andre/Documents/Git/SHP_MEC1515'

# Funções

## Compressor

* $Qc$: somatório da vazão demandada por cada elemento
* $K$: Fator que varia entre $1,2$ e $1,5$ (leva em consideração os vazamentos ou eventuais ampliações na planta pnemática)
* $I$: Coeficiente de inserção
    1. $T_t$: Tempo de funcionamento do compressor
    2. $T_s$: Tempo de parada do compressor

In [3]:
def coeficiente_de_insercao(Tt, Ts):
    return (Tt / (Tt + Ts))

def vazao_compressor(Qc, K, I):
    return (Qc * K / I)

## Atuadores (Cilindros)

* $D$: diâmetro do pistão (mm)
* $d$: diâmetro da haste (mm)
* $c$: curso do pistão (mm)
* $n$: número de ciclos por minuto (geralmente por ciclo entende-se uma fase de avanço e recuo)
* $pr$: pressão relativa de regime (MPa - 0,1 MPa = 1 bar)
* $pa$: pressão absoluta de regime (MPa - pa = pr + 0,1)
* $pan$: pressão absoluta de regime da câmara negativa (lado haste)
* $Q$: consumo do ar (litros / min)

In [4]:
# Consumo de ar de cilindros d0e simples efeito
def consumo_cilindros_simples(D, c, pa, n):
    return (D**2 * c * pa * n / 127000)

# Consumo de ar de cilindros de duplo efeito
def consumo_cilindros_duplo_efeito(D, d, c, pa, n, pressao_igual=False):
    if pressao_igual == False:
        return (c * n * (D**2 * pa + (D**2 - d**2) * pa * n / 127000))
    
    return (D**2 * c * pa * n / 63500)

## Reservatório

* $pr$: pressão relativa (MPa)
* $pa_0$: pressão absoluta do ar no estado livre (MPa)
* $V_0$: Volume de ar no estado livre ($m^3$)
* $pa_1$: Pressão absoluta do ar armazenado (MPa)
* $V_1$: Capacidade Total (Vazão total) em $m^3$

In [5]:
def capacidade_reservatorio(pr, V0, pa0=0.1):
    pa1 = pr + pa0
    return (V0 * pa0 / pa1)

# Dados do projeto

In [6]:
# Perimetro total 
perimetro_tubulacao = 2*18 + 7*13

# Quantidade extensoes que irao para 
#os atuadores das portas
extensoes = 20

## Preço Componentes

### Links

1. Pneu_3WaysValve: 
*  [Pneu_3WaysValve](https://produto.mercadolivre.com.br/MLB-3604089439-valvula-esfera-3-vias-l-1-inox-304-bsp-atuador-dupla-aco-_JM) (operation: 0 - 8 bar)

* [Pneu_3WaysValve](https://produto.mercadolivre.com.br/MLB-4657623390-valvula-esfera-3-vias-inox-rosca-34-bsp-atuador-dupla-aco-_JM#polycard_client=recommendations_vip-pads-up&reco_backend=vip-pads-up-experimental-a_marketplace&reco_model=ranker_entity_v2&reco_client=vip-pads-up&reco_item_pos=3&reco_backend_type=low_level&reco_id=c9efef8a-c2c5-4c45-af54-ee44fcfd7bd5&wid=MLB4657623390&sid=recos&is_advertising=true&ad_domain=VIPDESKTOP_UP&ad_position=4&ad_click_id=ZjJiNWZlYzMtYzczNy00YWE1LWJmNzUtN2EyZmU0NjRhM2Vh) (operation: 0 - 8 bar)

* [Pneu_3WaysValve](https://produto.mercadolivre.com.br/MLB-3604089439-valvula-esfera-3-vias-l-1-inox-304-bsp-atuador-dupla-aco-_JM#polycard_client=recommendations_vip-pads-up&reco_backend=vip-pads-up-experimental-a_marketplace&reco_model=ranker_entity_v2&reco_client=vip-pads-up&reco_item_pos=3&reco_backend_type=low_level&reco_id=7a01d7bc-9e61-44f7-9843-dfba6b192f44&wid=MLB3604089439&sid=recos&is_advertising=true&ad_domain=VIPDESKTOP_UP&ad_position=4&ad_click_id=NTMxZjg5ZjEtYTJhYi00NjRhLTliMmQtMmRhN2ZlZWFkZWFl) (operation: 0 - 8 bar)


2. Pneu_RegisterValve:
* [Pneu_RegisterValve](https://lfautomacao.com.br/products/valvula-esfera-mini-m-f-1-4?variant=45139439026498&country=BR&currency=BRL&utm_medium=product_sync&utm_source=google&utm_content=sag_organic&utm_campaign=sag_organic&utm_source=&utm_medium=&utm_campaign=&utm_content=&gad_source=1&gclid=Cj0KCQiA19e8BhCVARIsALpFMgEodeQtTI8wxnux3LrEpW_S0SSk5c01HWXtndNtISOookFEkKNXJu4aApdyEALw_wcB) (operation: 0 - 16 bar)

* [Pneu_RegisterValve](https://www.conecfit.com.br/produtos/mini-valvula-esfera-machoxfemea-3-8/?variant=955983600&pf=mc&gad_source=4&gclid=Cj0KCQiA19e8BhCVARIsALpFMgHHpWTi5bUf3u7U7_MfdZOl0nk18PJ7wiccGKI02TRtx7IfDSrl9JQaAgy0EALw_wcB) (operation: 0 - 6 bar)

3. Pneu_DrainValve:
* [Pneu_DrainValve](https://www.conecfit.com.br/produtos/valvula-solenoide-2vias-16bar-1-2220vca/?variant=956045908&pf=mc&gad_source=1&gclid=Cj0KCQiA19e8BhCVARIsALpFMgH9nbYfXipwIUmtqe3seNuw4dFSEb65lFWCyStoxmobvosGZTppys4aAuIbEALw_wcB) (operation: 0 - 16 bar)

* [Pneu_DrainValve](https://www.tauana.com.br/valvula-solenoide/valvula-solenoide-agua-ar-oleo/vs3513ba04nf-valvula-solenoide-latao-12-bsp?gad_source=1&gclid=Cj0KCQiA19e8BhCVARIsALpFMgEv1Oy0WxygwqY496xu89GEwQgiire1GvkWqmpHUlNtIL_W5flfiG4aAhD-EALw_wcB) (operation: 0 - 8 bar)

* [Pneu_DrainValve](https://www.conecfit.com.br/produtos/valvula-pneumatica-purgadoratimer-220vca/?variant=958126577&pf=mc&gad_source=1&gclid=Cj0KCQiA19e8BhCVARIsALpFMgEEGwH-6HRFA7sYwMP67AEbklsD9OuAfP8Mt3r5HZf32A0gOeyOmsgaArQDEALw_wcB) (operation: 0 - 8 bar)

4. Pneu_Actuator:
* [Pneu_Actuator](https://www.tauana.com.br/atuador-para-valvula/atuador-pneumatico/ap700l052-atuador-pneumatico-dupla-acao-22-nm?gad_source=1&gclid=Cj0KCQiA19e8BhCVARIsALpFMgHfuIpJ2g7SBsRbG4EAzFZ8WKbjOTolhN-ctR4cz8R02GeL9Tcwc9MaAvGUEALw_wcB) (operation: 0 - 8 bar)

* [Pneu_Actuator](https://www.conecfit.com.br/produtos/atuador-pneumatico-dupla-acao-52mm-eco/?variant=956637253&pf=mc&gad_source=1&gclid=Cj0KCQiA19e8BhCVARIsALpFMgEoM_N6CDG-12m5bxbcSEgEPI2X-iu7H3i08zyIQU80LkETztiFwocaAjKHEALw_wcB) (operation: 0 - 8 bar)

* [Pneu_Actuator](https://www.acasadoatuador.com.br/produtos/atuador-pneumatico-de-dupla-acao-at52-para-valvulas/?variant=853507488&pf=mc&gad_source=1&gclid=Cj0KCQiA19e8BhCVARIsALpFMgEJN5KnZAujnvTrAIXeZhmuYmlE3abaM1UO904K4i_RGo8jhEB7GGYaAlmTEALw_wcB) (operation: 0 - 6 bar)

In [8]:
Pneu_3WaysValve = np.mean([646.95, 710.97, 789.99])
Pneu_RegisterValve = np.mean([21.75, 26.26])
Pneu_DrainValve = np.mean([155.97, 209.90, 190.97])
Pneu_Actuator = np.mean([230., 288.12, 259.99])